# Native programming

TODO: introductory text for the notebook

In [1]:
# general imports
import numpy as np
import IPython

# AWS imports: Import Braket SDK modules
from braket.devices import Devices
import braket.experimental.autoqasm as aq
from braket.experimental.autoqasm.instructions import h, cnot, measure

TODO: introduce Bell state, why native programming might be important

In [2]:
@aq.main
def bell_state():
    h(0)
    cnot(0, 1)
    measure([0, 1])


bell_state_program = bell_state()
print(bell_state_program.to_ir())

OPENQASM 3.0;
qubit[2] __qubits__;
h __qubits__[0];
cnot __qubits__[0], __qubits__[1];
bit[2] __bit_0__ = "00";
__bit_0__[0] = measure __qubits__[0];
__bit_0__[1] = measure __qubits__[1];


TODO: explain why verbatim and physical qubits

In [3]:
@aq.main
def bell_state():
    with aq.verbatim():
        h("$0")
        cnot("$0", "$1")
    measure(["$0", "$1"])


bell_state_program = bell_state()
print(bell_state_program.to_ir())

OPENQASM 3.0;
pragma braket verbatim
box {
    h $0;
    cnot $0, $1;
}
bit[2] __bit_0__ = "00";
__bit_0__[0] = measure $0;
__bit_0__[1] = measure $1;


TODO: but what if I want to target this to a specific device? introduce `device` param

and try it here - but it doesn't work!

In [4]:
@aq.main(device=Devices.IonQ.Aria1)
def bell_state():
    with aq.verbatim():
        h("$0")
        cnot("$0", "$1")
    measure(["$0", "$1"])


try:
    bell_state_program = bell_state()
except Exception as e:
    print("ERROR:", e)

ERROR: The gate "h" is not a native gate of the target device "Aria 1". Only native gates may be used inside a verbatim block. The native gates of the device are: ['gpi', 'gpi2', 'ms']


TODO: introduce idea of custom gate definition, what we're doing here, explain what is in ionq_gates.py and how you could implement your own library of native gate implementations for any device.

In [5]:
IPython.display.Code(filename="ionq_gates.py")

import numpy as np
import braket.experimental.autoqasm as aq
from braket.experimental.autoqasm.instructions import gpi, gpi2, ms


@aq.gate
def h(q: aq.Qubit):
    gpi2(q, np.pi / 2)
    gpi(q, 0)


@aq.gate
def x(q: aq.Qubit):
    gpi(q, 0)


@aq.gate
def u(q: aq.Qubit, a: float, b: float, c: float):
    gpi2(q, a)
    gpi(q, b)
    gpi2(q, c)


@aq.gate
def rx(q: aq.Qubit, theta: float):
    u(q, np.pi / 2, theta / 2 + np.pi / 2, np.pi / 2)


@aq.gate
def ry(q: aq.Qubit, theta: float):
    u(q, np.pi, theta / 2 + np.pi, np.pi)


@aq.gate
def cnot(q0: aq.Qubit, q1: aq.Qubit):
    ry(q0, np.pi / 2)
    ms(q0, q1, 0, 0, np.pi / 2)
    rx(q0, -np.pi / 2)
    rx(q1, -np.pi / 2)
    ry(q0, -np.pi / 2)

TODO: now I can write the same program again, and it works!

In [6]:
from ionq_gates import h, cnot


@aq.main(device=Devices.IonQ.Aria1)
def bell_state():
    with aq.verbatim():
        h("$0")
        cnot("$0", "$1")
    measure(["$0", "$1"])


bell_state_program = bell_state()
print(bell_state_program.to_ir())

OPENQASM 3.0;
gate h q {
    gpi2(pi / 2) q;
    gpi(0) q;
}
gate u(a, b, c) q {
    gpi2(a) q;
    gpi(b) q;
    gpi2(c) q;
}
gate ry(theta) q {
    u(pi, theta / 2 + pi, pi) q;
}
gate rx(theta) q {
    u(pi / 2, theta / 2 + pi / 2, pi / 2) q;
}
gate cnot q0, q1 {
    ry(pi / 2) q0;
    ms(0, 0, pi / 2) q0, q1;
    rx(-(pi / 2)) q0;
    rx(-(pi / 2)) q1;
    ry(-(pi / 2)) q0;
}
pragma braket verbatim
box {
    h $0;
    cnot $0, $1;
}
bit[2] __bit_0__ = "00";
__bit_0__[0] = measure $0;
__bit_0__[1] = measure $1;


## Summary

TODO: summarize what we did!